In [3]:
from dataclasses import dataclass, field
import re

from pyprojroot import here

In [4]:
testInput = [
    'Valve AA has flow rate=0; tunnels lead to valves DD, II, BB',
    'Valve BB has flow rate=13; tunnels lead to valves CC, AA',
    'Valve CC has flow rate=2; tunnels lead to valves DD, BB,',
    'Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE',
    'Valve EE has flow rate=3; tunnels lead to valves FF, DD',
    'Valve FF has flow rate=0; tunnels lead to valves EE, GG',
    'Valve GG has flow rate=0; tunnels lead to valves FF, HH',
    'Valve HH has flow rate=22; tunnel leads to valve GG',
    'Valve II has flow rate=0; tunnels lead to valves AA, JJ',
    'Valve JJ has flow rate=21; tunnel leads to valve II'
]

In [5]:
@dataclass
class Valve:
    valveId: str
    flowRate: int
    children: list[str]


@dataclass
class Node:
    valve: Valve
    flowRate: int = 0
    pressureReleased: int = 0
    currentMinute: int = 1
    openedValves: set[str] = field(default_factory=set)
    valvesVisitedThisCycle: set[str] = field(default_factory=set)
    valveOpenedThisCycle: bool = False

In [6]:
def releaseMaxPressure(valves: dict[str, Valve], startingValve='AA', minutesAllowed=30):
    numValvesToOpen = len([valve for valve in valves.values() if valve.flowRate > 0])
    maxPressureReleased = 0
    pathsExplored = 0
    nodesVisited = 0
    stack = [Node(valves[startingValve])]

    while stack:
        node = stack.pop()
        nodesVisited += 1

        # time is up
        if node.currentMinute == minutesAllowed + 1:
            pathsExplored += 1
            continue

        # completed a cycle
        if node.valve.valveId in node.valvesVisitedThisCycle:

            # don't continue continue down this path
            if not node.valveOpenedThisCycle:
                continue

            # continue down this path
            else:
                node.valvesVisitedThisCycle = set()
                node.valveOpenedThisCycle = False

        # opened all valves
        if len(node.openedValves) == numValvesToOpen:
            pressureReleased = node.pressureReleased + (minutesAllowed - node.currentMinute) * node.flowRate
            maxPressureReleased = max(maxPressureReleased, pressureReleased)
            continue

        # pressure released during current minute
        node.pressureReleased += node.flowRate
        maxPressureReleased = max(maxPressureReleased, node.pressureReleased)

        # branch where we open valve during current minute
        if node.valve.flowRate > 0 and node.valve.valveId not in node.openedValves:
            openNode = Node(
                valve = node.valve,
                flowRate = node.flowRate + node.valve.flowRate,
                pressureReleased = node.pressureReleased,
                currentMinute = node.currentMinute + 1,
                openedValves = node.openedValves.union([node.valve.valveId]),
                valvesVisitedThisCycle = node.valvesVisitedThisCycle.union([node.valve.valveId]),
                valveOpenedThisCycle = True
            )
            stack.append(openNode)

        # branches where we travel to another valve during current minute
        for child in node.valve.children:
            childNode = Node(
                valve = valves[child],
                flowRate = node.flowRate,
                pressureReleased = node.pressureReleased,
                currentMinute = node.currentMinute + 1,
                openedValves = node.openedValves,
                valvesVisitedThisCycle = node.valvesVisitedThisCycle.union([node.valve.valveId]),
                valveOpenedThisCycle = node.valveOpenedThisCycle
            )
            stack.append(childNode)

    return nodesVisited, pathsExplored, maxPressureReleased

In [8]:
path = here('./16/input.txt')

with open(path, 'r') as fp:
    lines = fp.readlines()
    # lines = testInput
    data = [(*re.findall('([A-Z]{2}|\d+)(?!,|$)', line), re.findall('([A-Z]{2})(?=,|$)', line)) for line in lines]
    valves = {id: Valve(id, int(rate), children) for id, rate, children in data}

# (268028111, 111691104, 1737) # it worked but took 15m 12.4s
releaseMaxPressure(valves, minutesAllowed=30)

(268028111, 111691104, 1737)